In [ ]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from tensorflow import keras
import pandas as pd
from itertools import combinations
from tqdm import tqdm
from keras import backend as K

In [ ]:
one_hot = tf.keras.layers.CategoryEncoding(num_tokens=5, output_mode="one_hot")

def precision(y_true, y_pred):

    y_true = one_hot(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):

    y_true = one_hot(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [ ]:
validation_dir = '../5-7500-a/validation'

test_dir = '../5-7500-a/test'

train_dir = '../5-7500-a/train'

BATCH_SIZE = 32
IMG_SIZE = (224, 224)


train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 5250 files belonging to 5 classes.
Found 1150 files belonging to 5 classes.
Found 1125 files belonging to 5 classes.


In [ ]:
Xception = tf.keras.models.load_model('./xception.h5')
DenseNet121 = tf.keras.models.load_model('./densenet121.h5')
VGG16 = tf.keras.models.load_model('./vgg16.h5')
ResNet50V2 = tf.keras.models.load_model('./resnet_v2.h5')
InceptionResNetV2 = tf.keras.models.load_model('./inception.h5')
MobileNetV2 = tf.keras.models.load_model('./mobilenet.h5')
EfficientNetB0 = tf.keras.models.load_model('./efficientnet.h5')

Xception._name="Xception"
DenseNet121._name="DenseNet121"
VGG16._name="VGG16"
ResNet50V2._name = "ResNet50V2"
InceptionResNetV2._name = "InceptionResNetV2"
MobileNetV2._name = "MobileNetV2"
EfficientNetB0._name = "EfficientNetB0"

In [ ]:
def create_hard_voting_ensemble(models):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    models_outputs = []
    for model in models:
        models_outputs.append(model(inputs))
    if len(models_outputs) > 1:
        votes = [tf.equal(output, tf.reduce_max(output, axis=-1, keepdims=True)) for output in models_outputs] # get binary vectors, where 1 is for predicted class and other are zeros
        stacked_votes = tf.cast(tf.stack(votes, axis=1), models_outputs[0].dtype) # stack predictions
        voted_predictions = tf.reduce_sum(stacked_votes, axis=1)
        outputs = voted_predictions
    else:
        outputs = models_outputs[0]
    ansamble = keras.Model(inputs=inputs, outputs=outputs)

    base_learning_rate = 0.001
    ansamble.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy', precision, recall, f1])
    return ansamble


In [ ]:
#m = create_hard_voting_ensemble([Xception, MobileNetV2])
#m.summary()

In [ ]:
models = {'Xception': Xception, 'DenseNet121': DenseNet121, 'VGG16': VGG16, 'ResNet50V2': ResNet50V2, 'InceptionResNetV2': InceptionResNetV2,
          'MobileNetV2': MobileNetV2, 'EfficientNetB0': EfficientNetB0}

all_combs = []
for i in range(1, len(models)):
    all_combs.extend(list(combinations(models, i)))

ansambles = {}
for comb in all_combs:
    ansambles['-'.join(comb)] = comb

In [ ]:
print("All combinations: {}".format(len(ansambles)))

All combinations: 126


In [ ]:
result = pd.DataFrame({'name':[], 'test_loss': [], 'test_accuracy': [], 'test_precision': [], 'test_recall': [], 'test_f1': [], 'vallidation_accurasy': []})

for ansamble_name, model_names in tqdm(zip(ansambles, ansambles.values())):
    ansamble_models = [models[name] for name in model_names]
    ansamble = create_hard_voting_ensemble(ansamble_models)
    test_loss, test_acc, test_pres, test_recall, test_f1 = ansamble.evaluate(test_dataset)
    _, validation_acc, _, _, _ = ansamble.evaluate(validation_dataset)
    result.loc[len(result)] = [ansamble_name, test_loss, test_acc, test_pres, test_recall, test_f1, validation_acc]

0it [00:00, ?it/s]

36/36 [==============================] - 44s 944ms/step - loss: 0.3785 - accuracy: 0.8617 - precision: 0.8739 - recall: 0.8497 - f1: 0.8614


1it [01:44, 104.89s/it]

36/36 [==============================] - 44s 963ms/step - loss: 0.2046 - accuracy: 0.9270 - precision: 0.9384 - recall: 0.9235 - f1: 0.9307


2it [03:26, 103.05s/it]

...

124it [3:53:35, 122.96s/it]

36/36 [==============================] - 45s 978ms/step - loss: 0.5661 - accuracy: 0.9522 - precision: 0.7427 - recall: 0.9783 - f1: 0.8430


125it [3:55:33, 121.59s/it]

36/36 [==============================] - 45s 981ms/step - loss: 0.5376 - accuracy: 0.9539 - precision: 0.7781 - recall: 0.9774 - f1: 0.8655


126it [3:57:35, 113.14s/it]


In [ ]:
result.to_csv('./results.csv')

In [ ]:
result

,name,test_loss,test_accuracy,test_precision,test_recall,test_f1,vallidation_accurasy
0,Xception,0.249143,0.935111,0.942955,0.931424,0.937085,0.861739
1,DenseNet121,0.306656,0.876444,0.892356,0.870660,0.881223,0.926957
2,VGG16,0.483240,0.864889,0.870828,0.866319,0.868538,0.901739
3,ResNet50V2,0.401022,0.856000,0.863056,0.846875,0.854738,0.900870
4,InceptionResNetV2,0.514581,0.824889,0.833179,0.816493,0.824633,0.906087
...,...,...,...,...,...,...,...
121,Xception-DenseNet121-VGG16-ResNet50V2-MobileNe...,0.403429,0.950222,0.760453,0.987847,0.858641,0.953043
122,Xception-DenseNet121-VGG16-InceptionResNetV2-M...,0.446354,0.952889,0.734724,0.982292,0.839504,0.956522
123,Xception-DenseNet121-ResNet50V2-InceptionResNe...,0.447421,0.941333,0.762038,0.985243,0.858339,0.953913
124,Xception-VGG16-ResNet50V2-InceptionResNetV2-Mo...,0.466285,0.948444,0.726759,0.986111,0.835812,0.952174


In [ ]:
# find the best combination based on validation accuracy
ind = result.vallidation_accurasy.argmax() # find the maximum validation accuracy !
best_model = result.name.loc[ind]
print("The best model is: {}".format(best_model))

The best model is: Xception-DenseNet121-VGG16-MobileNetV2-EfficientNetB0


In [ ]:
# print test accuracy/loss for this model
print(result.loc[ind])

name                    Xception-DenseNet121-VGG16-MobileNetV2-Efficie...
test_loss                                                        0.396749
test_accuracy                                                    0.957333
test_precision                                                   0.777472
test_recall                                                      0.986979
test_f1                                                          0.868627
vallidation_accurasy                                             0.957391
Name: 103, dtype: object


In [ ]:
# create the best ensamble model again
ansamble_models = [models[name] for name in best_model.split("-")]
best_ansamble_model = create_hard_voting_ensemble(ansamble_models)
# save this model
best_ansamble_model.save('model_3.h5')

In [ ]:
# test saved model
from tensorflow.keras.models import load_model
loaded_model = load_model('model_3.h5', custom_objects={"precision": precision, 'recall': recall, 'f1': f1})
loaded_model.summary()

Model: "model_126"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_127 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Xception (Functional)          (None, 5)            20871725    ['input_127[0][0]']              
                                                                                                  
 DenseNet121 (Functional)       (None, 5)            7042629     ['input_127[0][0]']              
                                                                                                  
 VGG16 (Functional)             (None, 5)            14717253    ['input_127[0][0]']      